# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What are all the Python wrappers available for us to import?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'new_transport_interface']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [3]:
tiface_fut = new_transport_interface("0.0.0.0", 0)
tiface_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fa33631f3f0>()]>

Await the future to get the `TransportInterface`. Note: unlike in Rust, `await` isn't necessary to drive Future completion.

In [4]:
tiface = await tiface_fut
tiface

TransportInterface{ transport: Some(<Transport>),   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }

### Run the Transport in background
Here is the fun part :)

In [5]:
run_fut = tiface.run()
run_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fa33631d3d0>()]>

If the Future completed right away, it means something's fishy.

In [6]:
assert not run_fut.done()

Note the `transport: None` at the beginning of the following `repr` output. It is `None` because `run()` takes the `Transport`.

In [7]:
tiface

TransportInterface{ transport: None,   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }

## Transport Operation

### New Remote Endpoint

Make a transport command

In [8]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
chococat_endpoint

Endpoint(157.230.134.224:2016)

**SKIPPING** _The endpoint will timeout after 30 seconds of inactivity (30000ms)._

In [9]:
#new_endpoint_transport_cmd = TransportCmdW("newendpoint", endpoint=chococat_endpoint, timeout=30000)
#new_endpoint_transport_cmd

**SKIPPING** _Send the command to the Transport layer and get the response from it._

In [10]:
#await tiface.command_response(new_endpoint_transport_cmd)

### Send Packet to Remote

Create a `PacketSettings` wrapper. Note that the unique ID `tid` is optional, and duration is in milliseconds.

In [11]:
packet_settings = PacketSettingsW(5) # retry after 5ms
packet_settings

PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 5ms }

Make another transport command

Make a packet

In [12]:
action = RequestActionW(variant="connect", name="TransportDebugging", client_version="0.0.666")

In [13]:
pkt = PacketW(variant="request", sequence=1, response_ack=None, action=action, cookie=None)
pkt

Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "TransportDebugging", client_version: "0.0.666" } }

In [14]:


send_packets_cmd = TransportCmdW("sendpackets",
                                 endpoint=chococat_endpoint,
                                 packet_infos=[packet_settings],
                                 packets=[pkt])
send_packets_cmd

SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 5ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "TransportDebugging", client_version: "0.0.666" } }] }

SEND IT :)

In [15]:
await tiface.command_response(send_packets_cmd)

Get all available notifications from the Transport layer.

In [16]:
notifs = tiface.get_notifications()
notifs

Exception: transport notify channel was disconnected

Drain the notifications queue.

In [ ]:
notifs = tiface.get_notifications()
notifs